In [2]:
import tensorflow as tf

In [36]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, cross_validation, metrics
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [19]:
train_dat = pd.read_csv('train.csv')
test_dat = pd.read_csv('test.csv')
# Target encoding
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train_dat.target.values))
train_dat.target = lbl.transform(list(train_dat.target.values))
# Extract labels
train_labels = train_dat.target
# Drop labels
train_dat = train_dat.drop('target',axis=1)
# Drop id columns
train_dat = train_dat.drop('id',axis=1)
test_dat = test_dat.drop('id',axis=1)

In [20]:
n_classes = train_labels.nunique()
n_features = len(train_dat.columns)

In [28]:
Y = np.zeros( (len(train_labels), n_classes))
for i, v in enumerate(train_labels):
    Y[i,v] = 1

dev_cutoff = int( len(train_labels)*0.8 )
X_dev = train_dat.iloc[:dev_cutoff].as_matrix()
Y_dev = Y[:dev_cutoff]

X_test = train_dat.iloc[dev_cutoff:].as_matrix()
Y_test = Y[dev_cutoff:]

In [60]:
sess = tf.InteractiveSession()


# input layer
x = tf.placeholder("float", [None, n_features])
W = tf.Variable(tf.zeros([n_features, n_classes]))
b = tf.Variable(tf.zeros([n_classes]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder("float", [None, n_classes])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy)

In [62]:

epochs = 10
sss = cross_validation.StratifiedShuffleSplit(Y_dev, 100,
                             test_size=0.7, random_state=0)

# Train
tf.initialize_all_variables().run()
for train_index, test_index in sss:
    batch_xs, batch_ys = X_dev[train_index], Y_dev[train_index]
    train_step.run({x: batch_xs, y_: batch_ys})
   
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(accuracy.eval({x: X_test, y_: Y_test}))

#preds = y.eval({x: X_test, y_: Y_test})
#print(metrics.log_loss(Y_test, preds))

0.0
